In [8]:
import pdfplumber
import re
import json
import os
import PyPDF2


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

## METODI

In [2]:
file_path = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
# una semplice estrazione dei capitoli dal pdf
def extract_chapters_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"

    # Suddividiamo il testo nei capitoli basandoci su "CAPITOLO X"
    chapters = re.split(r'(Capitolo \d+)', text)
    
    structured_data = {}
    for i in range(1, len(chapters), 2):
        chapter_title = chapters[i].strip()
        chapter_content = chapters[i + 1].strip()
        structured_data[chapter_title] = chapter_content
    
    return structured_data

In [4]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )


In [133]:
def estrai_tecniche_e_macro(llm: AzureChatOpenAI, testo: str) -> tuple[dict, List[str]]:
    """
    Estrae le macrotecniche con descrizioni e le tecniche specifiche dal testo.
    
    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico
        testo (str): Il testo da analizzare
        
    Returns:
        tuple[dict, List[str]]: 
            - Dizionario delle macrotecniche con le loro descrizioni
            - Lista delle tecniche specifiche
    """
    # Prima estrazione: macrotecniche con descrizioni
    prompt_macro = """
    Analizza il seguente testo e identifica le macrotecniche culinarie.
    Restituisci un oggetto JSON con il nome della macrotecnica come chiave e la sua descrizione come valore.
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.
    
    Testo:
    {}
    """.format(testo)
    
    response_macro = llm.invoke(prompt_macro)
    content_macro = response_macro.content.strip()
    
    # Pulizia del JSON delle macrotecniche
    if content_macro.startswith('```') and content_macro.endswith('```'):
        content_macro = '\n'.join(content_macro.split('\n')[1:-1])
    if content_macro.startswith('json'):
        content_macro = content_macro[4:].strip()
    
    try:
        macrotecniche = json.loads(content_macro)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle macrotecniche:", content_macro)
        macrotecniche = {}
    
    # Seconda estrazione: tecniche specifiche
    prompt_tecniche = """
    Analizza il seguente testo e elenca SOLO le tecniche specifiche di preparazione (non le macrotecniche).
    Elenca solo i nomi delle tecniche, una per riga, senza punteggiatura o numerazione.
    Non aggiungere spiegazioni o altro testo.
    
    Testo:
    {}
    """.format(testo)
    
    response_tecniche = llm.invoke(prompt_tecniche)
    
    # Elaborazione delle tecniche specifiche
    tecniche = [
        riga.strip() 
        for riga in response_tecniche.content.split('\n') 
        if riga.strip() and not riga.startswith('-') and not riga[0].isdigit()
    ]
    
    # Rimuovi duplicati mantenendo l'ordine
    tecniche = list(dict.fromkeys(tecniche))
    
    return macrotecniche, tecniche

In [ ]:
def estrai_dettagli_tecniche_per_grafo(llm: AzureChatOpenAI, testo: str, tecniche: List[str]) -> dict:
    """
    Estrae informazioni strutturate per ogni tecnica dal testo, ottimizzate per GraphRAG.
    
    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico
        testo (str): Il testo originale da analizzare
        tecniche (List[str]): Lista delle tecniche trovate
        
    Returns:
        dict: Dizionario strutturato per costruzione del grafo
    """
    prompt_template = """
    Analizza la tecnica culinaria "{tecnica}" nel seguente testo e restituisci le informazioni in questo formato JSON strutturato:
    {{
        "tecnica": "nome della tecnica",
        "come_funziona": "breve descrizione del funzionamento della tecnica",
        "vantaggi": "vantaggi della tecnica",
        "svantaggi": "svantaggi della tecnica"
    }}
    Restituisci SOLO il JSON, senza formattazione markdown o altri decoratori.
    
    Testo da analizzare:
    {testo}
    """
    
    dettagli_grafo = {}
    
    for tecnica in tecniche:
        prompt = prompt_template.format(tecnica=tecnica, testo=testo)
        response = llm.invoke(prompt)
        
        try:
            # Assumendo che la risposta sia in formato JSON valido
            import json
            dettagli_grafo[tecnica] = json.loads(response.content)
        except json.JSONDecodeError:
            # Fallback se il JSON non è valido
            dettagli_grafo[tecnica] = {"error": "Formato non valido", "raw_content": response.content}
    
    return dettagli_grafo

In [10]:
def estrai_ordini(llm: AzureChatOpenAI, testo: str) -> Dict[str, str]:
    """
    Estrae gli ordini dal testo e li restituisce come dizionario JSON.
    
    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico
        testo (str): Il testo da analizzare
        
    Returns:
        Dict[str, str]: Dizionario con nome dell'ordine come chiave e descrizione come valore
    """
    prompt = """
    Analizza il seguente testo e identifica gli Ordini presenti.
    Restituisci un oggetto JSON dove la chiave è il nome dell'Ordine e il valore è la sua descrizione.
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.
    
    Testo:
    {}
    """.format(testo)
    
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()
    
    try:
        ordini = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON degli Ordini:", content)
        ordini = {}
    
    return ordini

In [15]:
def estrai_licenze(llm: AzureChatOpenAI, testo: str) -> Dict[str, str]:
    """
    Estrae le licenze dal testo, combinando categoria e livello nel nome.
    
    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico
        testo (str): Il testo da analizzare
        
    Returns:
        Dict[str, str]: Dizionario con nome della licenza (categoria + livello) come chiave 
                       e descrizione come valore
    """
    prompt = """
    Analizza il seguente testo e identifica tutte le licenze presenti.
    Per ogni categoria di abilità (es. Psionica, Temporale), estrai tutti i livelli disponibili.
    
    Restituisci un oggetto JSON dove:
    - La chiave è nel formato "NomeCategoria livello X" (es. "Psionica livello 0")
    - Il valore è la descrizione completa di quella licenza specifica
    
    Assicurati di:
    - Includere tutti i livelli per ogni categoria
    - Mantenere le descrizioni complete e pertinenti
    - Non omettere informazioni importanti
    
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.
    
    Testo:
    {}
    """.format(testo)
    
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()
    
    try:
        licenze = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle licenze:", content)
        licenze = {}
    
    return licenze

In [18]:
# Funzione per pulire il database
def pulisci_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")


def aggiungi_macrotecniche(tx, nome, descrizione):
    query = """
    MERGE (m:Macrotecnica {nome: $nome})
    SET m.descrizione = $descrizione
    """
    tx.run(query, nome=nome, descrizione=descrizione)

# Funzione per aggiungere le tecniche
def aggiungi_tecniche(tx, nome, come_funziona, vantaggi, svantaggi):
    query = """
    MERGE (t:Tecnica {nome: $nome})
    SET t.come_funziona = $come_funziona,
        t.vantaggi = $vantaggi,
        t.svantaggi = $svantaggi
    """
    tx.run(query, nome=nome, come_funziona=come_funziona, vantaggi=vantaggi, svantaggi=svantaggi)

def aggiungi_ordine(tx, nome: str, descrizione: str):
    """
    Aggiunge un ordine al database Neo4j.
    
    Args:
        tx: Transazione Neo4j
        nome (str): Nome dell'ordine
        descrizione (str): Descrizione dell'ordine
    """
    query = """
    MERGE (o:Ordine {nome: $nome})
    SET o.descrizione = $descrizione
    """
    tx.run(query, nome=nome, descrizione=descrizione)

def aggiungi_licenza(tx, nome: str, descrizione: str):
    """
    Aggiunge una licenza al database Neo4j.
    
    Args:
        tx: Transazione Neo4j
        nome (str): Nome della licenza (categoria + livello)
        descrizione (str): Descrizione della licenza
    """
    query = """
    MERGE (l:Licenza {nome: $nome})
    SET l.descrizione = $descrizione
    """
    tx.run(query, nome=nome, descrizione=descrizione)

## MAIN

In [6]:
# Utilizzo:
llm = crea_llm(api_key, api_base)

# Leggiamo il PDF e estraiamo i capitoli
pdf_chapters = extract_chapters_from_pdf(file_path)
capitolo_1 = pdf_chapters["Capitolo 1"]
capitolo_2 = pdf_chapters["Capitolo 2"]

### CAPITOLO 1 

In [16]:
licenze = estrai_licenze(llm, capitolo_1)

In [17]:
print(licenze)

{'Psionica livello 0': 'Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.', 'Psionica livello I': 'lettura pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti', 'Psionica livello II': 'manipolazione della probabilità, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dell’universo', 'Psionica livello III': 'capacità di donare la coscienza e l’intelletto ad oggetti, manipolazione della realtà circoscritta a stanze, teletrasporto senza errore in qualsiasi dimensione temporale, comunione con entità di altri piani', 'Psionica livello IV': 'proiezione astrale, riscrittura di realtà circoscritta a piccole nazioni o asteroidi', 'Psionica livello V': 'riscrittura di realtà di intere linee temporali o galassie. Questo livello è equivalente al Grado di influenza di livello tecnologico III (LTK III)', 'Temporale livello I': 'effetti tem

In [19]:
# Inserimento dati nel database
with driver.session() as session:
    for licenza, descrizione in licenze.items():
        session.execute_write(aggiungi_licenza, licenza, descrizione)

print("Licenze aggiunti con successo!")

Ordini aggiunti con successo!


### CAPITOLO 2 

In [11]:
# Estrai ordini dal testo
ordini = estrai_ordini(llm, capitolo_2)

In [12]:
print(ordini)

{'Ordine della Galassia di Andromeda': 'Cari pionieri del gusto cosmico, preparatevi a una sfida da veri fuoriclasse: cucinare per gli abitanti di Andromeda! Qui non parliamo solo di tecnica stellare, ma di pura diplomazia culinaria intergalattica. La particolarità? Ogni piatto devʼessere rigorosamente privo di lattosio — non un goccio, nemmeno lʼombra di una molecola originaria della Via Lattea. Se pensate che il formaggio sia lʼunica via per la felicità, è tempo di ripensare il vostro approccio. Imparerete a creare sapori complessi senza latte, burro o panna, conquistando palati alieni con alternative che sfidano le leggi della gastronomia universale.', 'Ordine dei Naturalisti': 'Per gli adepti dellʼOrdine dei Naturalisti, la cucina è molto più che creatività: è un atto di riverenza verso la natura. Qui si celebra lʼessenza primordiale di ogni ingrediente, custodendone la purezza come un prezioso segreto galattico. Nessuna trasformazione drastica, niente manipolazioni invasive: solo 

In [14]:
# Inserimento dati nel database
with driver.session() as session:
    for ordine, descrizione in ordini.items():
        session.execute_write(aggiungi_ordine, ordine, descrizione)

print("Ordini aggiunti con successo!")

Ordini aggiunti con successo!


### CAPITOLI 3, 4, 5

In [ ]:
capitolo_3 = pdf_chapters["Capitolo 3"]
capitolo_4 = pdf_chapters["Capitolo 4"]
capitolo_5 = pdf_chapters["Capitolo 5"]

macrotecniche, tecniche = estrai_tecniche_e_macro(llm, capitolo_5)

# Per visualizzare i risultati:
print("\nMacrotecniche:")
for macro, descrizione in macrotecniche.items():
    print(f"\n{macro}:")
    print(descrizione)

print("\nTecniche specifiche:")
for tecnica in tecniche:
    print(f"- {tecnica}")

In [ ]:
dettagli_per_grafo = estrai_dettagli_tecniche_per_grafo(llm, capitolo_5, tecniche)

# Inserimento dati nel database
with driver.session() as session:
    for macro, descrizione in macrotecniche.items():
        session.execute_write(aggiungi_macrotecniche, macro, descrizione)

print("Macrotecniche aggiunte con successo!")

# Inserire le tecniche nel database
with driver.session() as session:
    for nome, dettagli in dettagli_per_grafo.items():
        session.execute_write(
            aggiungi_tecniche, nome, dettagli["come_funziona"], dettagli["vantaggi"], dettagli["svantaggi"]
        )

print("Tecniche aggiunte con successo!")